In [9]:
import serial
import pandas as pd
from datetime import datetime, timedelta
import os
from playsound import playsound

# Configure the serial port and baud rate
ser = serial.Serial('/dev/cu.usbserial-1110', 115200, timeout=1)

# Load or create the attendance DataFrame
try:
    df = pd.read_csv('attendance.csv')
except FileNotFoundError:
    df = pd.DataFrame(columns=['QR Code'])

# Define a function to determine the current week number
def get_week_number(date):
    start_date = datetime(2024, 3, 20)  # Change to the start date of your term YYYY-MM-DD
    return ((date - start_date).days // 7) + 1

# Specify the path for the CSV file
output_path_csv = os.path.join(os.getcwd(), 'attendance.csv')
print(f"CSV file will be saved to: {output_path_csv}")

# Variables for debounce mechanism
last_qr_code = None
last_scan_time = datetime.min
debounce_time = timedelta(seconds=10)  # Adjust debounce time as needed

try:
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line and line.startswith("Found '") and line.endswith("'"):
            qr_code = line[7:-1]  # Extract the QR code content
            current_time = datetime.now()

            # Debugging output for debounce mechanism
            print(f"Current QR Code: {qr_code}")
            print(f"Last QR Code: {last_qr_code}")
            print(f"Current Time: {current_time}")
            print(f"Last Scan Time: {last_scan_time}")
            print(f"Time Since Last Scan: {current_time - last_scan_time}")

            # Check for debounce
            if qr_code != last_qr_code or (current_time - last_scan_time) > debounce_time:
                last_qr_code = qr_code
                last_scan_time = current_time
                print(f"Processing QR code: {qr_code}")  # Debugging statement

                timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S')
                week_number = get_week_number(current_time)
                week_col = f'Week {week_number}'
                print(f"Received QR code: {qr_code}")  # Debugging statement

                if qr_code not in df['QR Code'].values:
                    new_entry = pd.DataFrame({'QR Code': [qr_code]})
                    df = pd.concat([df, new_entry], ignore_index=True)
                    print(f"Added new entry: {qr_code}")  # Debugging statement
                
                if week_col not in df.columns:
                    df[week_col] = 'Absent'  # Default to 'Absent' for all initially
                    print(f"Added new column for {week_col}")  # Debugging statement
                
                df.loc[df['QR Code'] == qr_code, week_col] = 'Present'
                print(f"Marked {qr_code} as present for {week_col}")  # Debugging statement
                
                print("Saving to CSV...")  # Debugging statement before saving
                df.to_csv(output_path_csv, index=False)
                print("CSV file saved.")  # Debugging statement after saving
                print(f'Logged: {qr_code} at {timestamp} for {week_col}')
                
                # Play sound
                playsound('/Users/jbs939/Documents/StudioRepo/Attenderly/spaceShip.mp3')
            else:
                print(f"Debounce active: Ignoring QR code {qr_code}")
except KeyboardInterrupt:
    print("\nInterrupt received. Saving CSV file and exiting...")
    df.to_csv(output_path_csv, index=False)
    print("CSV file saved.")

CSV file will be saved to: /Users/jbs939/Documents/StudioRepo/Attenderly/attendance.csv
Current QR Code: T7GHBY
Last QR Code: None
Current Time: 2024-07-20 23:20:40.544308
Last Scan Time: 0001-01-01 00:00:00
Time Since Last Scan: 739086 days, 23:20:40.544308
Processing QR code: T7GHBY
Received QR code: T7GHBY
Marked T7GHBY as present for Week 18
Saving to CSV...
CSV file saved.
Logged: T7GHBY at 2024-07-20 23:20:40 for Week 18
Current QR Code: T7GHBY
Last QR Code: T7GHBY
Current Time: 2024-07-20 23:20:46.204397
Last Scan Time: 2024-07-20 23:20:40.544308
Time Since Last Scan: 0:00:05.660089
Debounce active: Ignoring QR code T7GHBY
Current QR Code: T7GHBY
Last QR Code: T7GHBY
Current Time: 2024-07-20 23:20:47.110287
Last Scan Time: 2024-07-20 23:20:40.544308
Time Since Last Scan: 0:00:06.565979
Debounce active: Ignoring QR code T7GHBY
Current QR Code: T7GHBY
Last QR Code: T7GHBY
Current Time: 2024-07-20 23:20:47.336066
Last Scan Time: 2024-07-20 23:20:40.544308
Time Since Last Scan: 0:00